This notebook follows the recommendations in P. Olofsson et al. (2014) to create a stratified sampling design per map class (page 52 sec. 5). 
The goal is to create a sampling design by exploring how accuracy metrics change according to different sample sizes and sample allocations per class. 
The metrics used to investigate sampling design are overall and class-specific accuracies, and confidence intervals for each accuracy and area parameter.

In [1]:
import os
import numpy as np
import pandas as pd

### Determining the sample size (Olofsson et al. 2014 - sec. 5.1.1)

For stratified random sampling Cochran (1977 Eq 5.25) suggests the following sample size formula (Olofsson et al. 2014 Eq. 13):

$$n \approx \left( \frac{\sum W_iS_i}{S(\hat{O})}\right)^2,$$

where

- $S(\hat{O})$ is the standard error of the estimated overall accuracy $\hat{O}$ that we would like to achieve,

- $W_i$ is the fraction of the map labeled as class $i$, and

- $S_i = \sqrt{U_i(1-U_i)}$ is the standard deviation of class $i$.

In the standard deviation formula, $U_i$ is the user's accuracy for class $i$, i.e. the precision for each class: TP/(TP+FP).

For determining sample size using the previous equation, we then have to estimate the user's accuracy for each class, $U_i$, and the std. error of the estimated overall accuracy, $S(\hat{O})$. The class fractions $W_i$ are not estimated since they come from counting the number of pixels in each class in the map. 

To estimate $U_i$ (and thus obtain the standard deviations) we can use the user's accuracies obtained from applying the model on the test set. Take into account that estimating $U_i$ from the model testing might still be optimistic. 

Recall that the square root of the estimated variance results in the standard error of the estimator. For example, in the case of the estimated overall accuracy of the map $\hat{O}$ we have that $\hat{S}(\hat{O}) = \sqrt{\hat{V}(\hat{O})}$ (Olofsson et al. 2014 Eq. 5). The standard error is used to get confidence intervals for the estimated statistic: the 95% confidence interval is estimated as $\hat{O} \pm 1.96 \hat{S}(\hat{O}) = \hat{O} \pm 1.96 \sqrt{\hat{V}(\hat{O})}$.

To find a value of $S(\hat{O})$ we feel comfortable with, we need to take into account the following: suppose we want an overall accuracy ((TP + TN)/(P+N)) of 0.9, meaning 90% of the pixels were correctly classified. A std. error of 0.01 would mean that the 95% confidence interval around the estimated overall accuracy is $0.9 \pm 0.01*1.96 = 0.9 \pm 0.0196$, so OA is between 88% and 92%. If we increase std error to 0.02, the 95% confidence interval for the estimated OA woult be between 86% and 94%. So small changes in $S(\hat{O})$ will alter the width of the confidence interval.

### Import data: Pixels per class in map of SB coast

In [2]:
# ---------------------------------------------
# --------------- PARAMETER -------------------
df = pd.read_csv('salt13_p30_total_pixel_counts.csv')
df

,n_other_veg,n_ice,n_nonveg
0,120173466,5981423,188071487


In [3]:
# ---------------------------------------------
# --------------- PARAMETERS ------------------
# standard error for all the points
std_error = 0.01

# estimates of user's accuracies TP/(TP+FP)
# classes are: [other vegetation, iceplant, non_vegetation]
U = [0.8184, 0.8468, 0.9]
# ---------------------------------------------

# fraction of pixels with a given class in total pixels
total_pix = sum(df.iloc[0,])
pix_prop = [n/total_pix for n in list(df.iloc[0,])]

# standard deviation of user's accuracies
# Cochran, 1977, Eq (5.55)
stdv = [ np.sqrt(u*(1-u)) for u in U]

numerator = sum([ x*y for x,y in zip(pix_prop, stdv)])

sample_size = (numerator/std_error)**2
sample_size

1114.557709288247

### Determine sample allocation per class (Olofsson et al., 2014 - sec. 5.1.2)

Once the overall sample size is chosen, we can determine how many sample points to allocate in each class. s
Roughly speaking, there are two poles for the sample allocation: 

(1) propotional: allocate sample sizes proportionally to the area covered by each class, and

(2) equal: divide the sample size equally among classes. 

The tradeoffs are that proportional allocation will have few samples in rare classes and thus will give imprecise estimates of user's accuracy of these rare classes. 
On the other hand, according to Olofsson et al., equal allocation "is not optimized for estimating area and overall accuracy". 
A suggested middle point is to first allocate a fixed sample size of 50-100 to rare classes, calibrating the sample size according to what standard error we would want to achieve based on the assumed user's accuracies for those classes. 
After rare classes have a specified sample allocation, then we proportionally distribute the remaining points from the sample among the remaining classes. The the estimated std. errors and accuracies can be computed using the sample allocation per class.

In [4]:
def sample_allocation(n_pix_df, fixed_classes, fixed_n, sample_size):
    """ sample allocation per class combining fixed + proportional allocation
    
    Allocates classes in fixed_classes a fixed number of sample points and 
    distributes the remaing pts proportionally across the rest of the classes
    
    Parameters
    ----------
    n_pix_df : pandas.DataFrame
        one-row data frame with the number of pixels per class. 
        columns must be names of classes
        
    fixed_classes : list
        subset of n_pix_df.columns. these are the classes that will get a 
        fixed allocation of points.
    
    fixed_n : int
        the number of samples to allocate to indicated classes
    
    sample_size : int
        number of points to be distributed among classes
        see Oloffson et al., 2014, Eq. 13
    """
    
    # non-fixed allocation classes which will get proportional sample allocation 
    prop_classes = list(set(n_pix_df.columns) - set(fixed_classes))

    # add total of pixels in non-fixed allocation classes
    d = n_pix_df[prop_classes].sum(axis=1).iloc[0]

    # get proportions for each non-fixed class
    props = n_pix_df[prop_classes]/d

    # remaining points after allocating to fixed classes
    remain = sample_size-(fixed_n*len(fixed_classes))

    samples = []
    for col in n_pix_df.columns:
        if col in fixed_classes:
            samples.append(fixed_n)
        else:
            samples.append(int(props[col][0]*remain))
    return samples

# --------------------------------------------------------------------------------------
def strat_stderror(U, strat_sample):
    """ estimated standard error of estimated user's accuracies (U) per class 
    
    See Oloffson et al., 2014, Eq. 6.
    
    Parameters:
    ----------
    U : list
        a list with the estimated user's accuracies per class, each is a number in [0,1]
        
    strat_sample: list
        list of integers indicating how many points to sample from reach class
    
    Return:
    -------
    A list with the std. errors for each class based on U and the stratified sample.
    
    """
    
    return [ np.sqrt(u*(1-u)/(n-1)) for u,n in zip(U, strat_sample) ]
# --------------------------------------------------------------------------------------
def confidence_intervals(U, strat_sample):
    """ radius of 95% conf. interval around estimated user's accuracy for each class
    
    Parameters
    ----------
    U : list
        a list with the estimated user's accuracies per class, each is a number in [0,1]
        
    strat_sample: list
        list of integers indicating how many points to sample from reach class
    
    Return:
    -------
        a list with the radius of the 95% confidence interval (as a percentage and 
        rounded to two decimal places) for the estimated user's accuracies

    """
    
    se = strat_stderror(U, strat_sample)
    return [np.round(196*x,2) for x in se]
    

In [5]:
# Distributing sample among classes

conf_intrs =[]
strat_samples = []
strat_title = []

# ---------------------------------------------
strat_title.append('equal')
sample_equal = [sample_size/len(df.columns) for i in range(0,len(df.columns))]
strat_samples.append(sample_equal)
conf_intrs.append(confidence_intervals(U, sample_equal))

# ---------------------------------------------
# vegetation and iceplant get equal allocations
fixed_classes = ['n_other_veg','n_ice']

for n in [200,150, 140, 130, 120, 100, 90,75]:
    strat_title.append(str(n))
    sample = sample_allocation(df, fixed_classes, n, sample_size)
    strat_samples.append(sample)
    conf_intrs.append(confidence_intervals(U, sample))
    
# ---------------------------------------------
# only iceplant gets fixed allocation
fixed_classes = ['n_ice']

for n in [200,150, 140, 130, 120, 100, 90,75]:
    strat_title.append(str(n))
    sample = sample_allocation(df, fixed_classes, n, sample_size)
    strat_samples.append(sample)
    conf_intrs.append(confidence_intervals(U, sample))    
    
# ---------------------------------------------

strat_title.append('prop')
sample_prop = [sample_size*x for x in pix_prop]
strat_samples.append(sample_prop)
conf_intrs.append(confidence_intervals(U, sample_prop))


In [6]:
strat_df = pd.DataFrame(strat_samples).T
strat_df.columns  = strat_title
strat_df

,equal,200,150,140,130,120,100,90,75,200,150,140,130,120,100,90,75,prop
0,371.519236,200.0,150.0,140.0,130.0,120.0,100.0,90.0,75.0,356.0,376.0,379.0,383.0,387.0,395.0,399.0,405.0,426.254042
1,371.519236,200.0,150.0,140.0,130.0,120.0,100.0,90.0,75.0,200.0,150.0,140.0,130.0,120.0,100.0,90.0,75.0,21.216046
2,371.519236,714.0,814.0,834.0,854.0,874.0,914.0,934.0,964.0,558.0,588.0,594.0,600.0,606.0,619.0,625.0,634.0,667.087621


In [7]:
conf_intrs_df = pd.DataFrame(conf_intrs).T
conf_intrs_df.columns  = strat_title
conf_intrs_df

,equal,200,150,140,130,120,100,90,75,200,150,140,130,120,100,90,75,prop
0,3.93,5.36,6.19,6.41,6.65,6.93,7.59,8.01,8.78,4.01,3.90,3.89,3.87,3.85,3.81,3.79,3.76,3.66
1,3.67,5.00,5.78,5.99,6.22,6.47,7.10,7.48,8.21,5.00,5.78,5.99,6.22,6.47,7.10,7.48,8.21,15.70
2,3.05,2.20,2.06,2.04,2.01,1.99,1.95,1.93,1.89,2.49,2.43,2.41,2.40,2.39,2.37,2.35,2.34,2.28
